# 1. Package Installation

In [5]:
#@title Requirements
%%writefile requirements.txt
peft
fire
accelerator
transformers
datasets
pyarrow
galore-torch
py7zr

Overwriting requirements.txt


In [6]:
#@title Install Packages
%%capture
!pip install --no-cache -r requirements.txt

In [6]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

# 2. Load Model


In [ ]:
#@title Get peft model from huggingface
#@markdown Colab 고용량 Ram CPU에서 가능한 범위 ~8B(테스트 중)

import fire
import torch
from peft import AutoPeftModelForCausalLM
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from random import randint

base_model_id = "google/gemma-7b" # @param ["meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-70B-Instruct", "mistralai/Mistral-7B-Instruct-v0.3", "Qwen/Qwen2-7B-Instruct", "google/gemma-7b", "MLP-KTLim/llama-3-Korean-Bllossom-8B"] {allow-input: true}

peft_model = AutoModelForCausalLM.from_pretrained(base_model_id)

# adapter configuration
lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights="gaussian", #"gaussian", "pissa", "pissa_niter_{n}", "loftq", False
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    inference_mode=False,
    use_dora=False,
)

peft_model.add_adapter(lora_config, adapter_name="adapter_1")

tokenizer = AutoTokenizer.from_pretrained(base_model_id)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

#3. Load Dataset

In [ ]:
#@title Load data From huggingface datasets
#@markdown summary task에 대해 우선적으로 실험
from datasets import load_dataset

dataset_path = "Samsung/samsum" # @param ["Samsung/samsum", "emozilla/soda_synthetic_dialogue", "frcp/summary-alpaca-v01"] {allow-input: true}

dataset = load_dataset(
  dataset_path,
  revision="main"  # tag name, or branch name, or commit hash
)

#4. Train

In [ ]:
#@title Start Training
#@markdown transformers trainer 이용, 추후 lightning 으로 이전 가능
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="llm_output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()